In [1]:
import numpy as np

# Part 1

Need to find total of answers to cephalopod worksheet.

### Data Input and Conversion

Import data and split into two arrays: the last row as the OPERATORS, and the main body as the OPERANDS.

Each problem is separated by a single column of whitespace, so forcing the import of whitespace for further processing will be key here.

In [2]:
# import data
data = np.loadtxt(
    fname='Input6.txt',
    dtype='O',
    delimiter='@'
)

In [3]:
data.shape

(5,)

Forced import of whitespace by setting delimiter to @ (arbitrary symbol that is not in dataset).

In [4]:
data[0][:50]

'23  721    8 853 826 24   5   857 5   75 14 8246 1'

### Helpers

In [5]:
def split_string(string: str, indices: list, remove_index_char: bool = True) -> list:
    """
    Split a string at the given positions, with option to remove the character at the split point.
    """
    # instantiate output array of split strings
    output_array = []
    
    # instantiate starting index 
    start_index = 0
    
    # traverse indices of splits
    for current_index in indices:
        # append each split to output
        output_array.append(string[start_index:current_index])

        # update indices, with option to remove the character at the split point
        start_index = current_index + remove_index_char
    
    # finally, add last split
    output_array.append(string[start_index:])
    
    return output_array

In [6]:
def split_array_by_common_char(input_array: np.array, char: str = ' ') -> np.array:
    """For a given input array, return output array split blank columns, if any.
    
    "Blank" means 1 whitespace character, and "blank column" means 1 whitespace character in the same position across all rows of data.
    
    Input is a 1d array of row data that would yield a 2d array if rows were vertically stacked.
    """
    # instantiate array of indices
    indices = []
    
    # instantiate output array of split strings
    output_array = np.empty_like(input_array)
    
    # number of rows
    rows = len(input_array)
    
    # number of columns
    cols = len(input_array[0])
    
    # traverse each char of each element, 'columns'
    for i in range(cols):
        if (np.array([j[i] for j in input_array]) == char).all():  # check if all element are whitespace
            indices.append(i)  # if so, make note of index
    
    # once list of indices completed, split the strings by index
    for k in range(rows):
        output_array[k] = split_string(input_array[k], indices, True)
        
    return output_array

In [7]:
# process raw data to yield correclty split strings
data_split = split_array_by_common_char(data)

In [8]:
# ===== OPERANDS ARRAY =====

# convert raw data to 2D integer ndarray

# instantiate ndarray of 64bit unsigned ints
operands = np.zeros(
    shape=(data_split.shape[0] - 1, len(data_split[0])),
    dtype=np.int64
)

# iterate through raw data to populate array
for i in range(data_split.shape[0] - 1):
    for j in range(len(data_split[0])):
        operands[i, j] = np.int64(data_split[i][j])

In [9]:
operands[:,:5]

array([[  23,  721,    8,  853,  826],
       [ 572, 9786,   58,  295,  534],
       [ 944, 3137,  229,  229,   81],
       [ 631, 1237,  217,  777,   21]])

In [10]:
# ===== OPERATORS ARRAY =====

# convert raw data to 1D string ndarray

# instantiate ndarray
operators = np.zeros(
    shape=(len(data_split[0]),),
    dtype='O'
)

# iterate through raw data to populate array
for i in range(len(data_split[0])):
    operators[i] = data_split[-1][i].strip()

In [11]:
operators[:5]

array(['*', '+', '+', '*', '*'], dtype=object)

### Helpers

In [12]:
def calculate_problem(input_operands: np.array, operator: str) -> int:
    """For a given operand array and operator, evaluate and return solution."""
    # instantiate solution
    solution = 0
    
    # multiplication
    if operator == '*':
        return np.prod(input_operands)
    
    # addition
    elif operator == '+':
        return np.sum(input_operands)

### Process Worksheet

In [13]:
def problem_worksheet_total(operands: np.array, operators: np.array) -> int:
    """Return total of worksheet problems, given operand and operator inputs."""
    # instantiate output
    grand_total = np.int64(0)
    
    # loop through list of operators...
    for i,j in enumerate(operators):
        grand_total += calculate_problem(operands[:,i], j)  # add problem solution to grand total

    return grand_total

In [14]:
problem_worksheet_total(operands, operators)

np.int64(6957525317641)

# Part 2

Updated method of reading cephalopod math: right-to-left (non-material) and each number from top to bottom.

### Helpers

In [15]:
def stitch_strings_to_int(numbers: list) -> np.int64:
    """Given a list of integer strings, stitch back to one integer.
    
    E.g. ['4', '2', '1'] -> 421
    """
    # output integer
    output = ''
    
    # error check argument
    if (len(numbers) < 1):
        return 0

    # stitch strings together
    for i in numbers:
        output += i
    
    # return integer with whitespace stripped
    return np.int64(output.strip())

In [16]:
# ===== OPERANDS (IN STRING FORM) =====

# convert raw data to 2D integer ndarray

# instantiate ndarray
operands_str = np.zeros(
    shape=(data_split.shape[0] - 1, len(data_split[0])),
    dtype='O'
)

# iterate through raw data to populate array
for i in range(data_split.shape[0] - 1):
    for j in range(len(data_split[0])):
        operands_str[i, j] = data_split[i][j]

In [17]:
# check char length of each column is consistent through all rows
_data_validation = []

# across all columns...
for i in range(operands_str.shape[1]):
    # ... check if each row is same length as first row i.e. all the same
    if (np.array([len(j) for j in operands_str[:, i]]) == len(operands_str[0, i])).all():
        _data_validation.append(True)
    else:
        _data_validation.append(False)
        
# final check if data is consistent
print("OK!") if (np.array(_data_validation) == True).all() else print("CHECK")

OK!


### Process Worksheet (New Method)

In [18]:
def problem_worksheet_total_new(input_operands: np.array, input_operators: np.array) -> np.int64:
    """Return total of worksheet problems, given operand and operator inputs.
    
    Operand array is 2d array of strings of integers/whitespace.
    
    New method of parsing numbers in use.
    """
    # instantiate output
    grand_total = np.int64(0)
    
    # working variable for columns width
    col_width = 0
    
    # working list for parsed numbers
    parsed_ints = []
    
    # using the list of operators as the counter ...
    for i, j in enumerate(input_operators):
        
        # number of chars in column
        col_width = len(input_operands[:, i][0])
        
        # ... loop through each char position in the column
        # parsing the operands with the new column-wise method
        for k in range(1, col_width + 1):
            parsed_ints.append(
                stitch_strings_to_int(
                    [l[-k] for l in input_operands[:, i]]  # list of integer strings, read left-to-right
                )
            )
        
        grand_total += calculate_problem(parsed_ints, j)  # add problem solution to grand total
        
        # reset parsed ints
        parsed_ints = []

    return grand_total  # np.uint64(grand_total)

In [19]:
problem_worksheet_total_new(operands_str, operators)

np.int64(13215665360076)